In [44]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, random_split

y = 2 + 7 * sin(3 * x)

y = a + b * sin(c * x)

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [46]:
np.random.seed(42)
# для тренировочных данных
sz = 100
x = np.random.rand(sz, 1)
y = 2 + 7 * np.sin(3 * x) + np.random.randn(sz, 1)

In [47]:
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()
dataset = TensorDataset(x_tensor, y_tensor)
train_dataset, val_dataset = random_split(dataset, [80, 20])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=True)

In [48]:
# Создаем нелинейную модель
class ManualNonLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.c = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x):
        return self.a + self.b * torch.sin(3 * x)

In [49]:
# Параметры для обучения
n_epochs = 10000
lr = 0.1
loss_fn = nn.MSELoss(reduction="mean")
model = ManualNonLinearRegression()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [50]:
# Функция для создания шага обучения
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss
    return train_step

In [51]:
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    # обучение
    for x_train_batch, y_train_batch in train_loader:
        x_train_batch.to(device)
        y_train_batch.to(device)
        loss = train_step(x_train_batch, y_train_batch)

    # валидация
    with torch.no_grad():
        for x_val_batch, y_val_batch in val_loader:
            model.eval()
            x_val_batch.to(device)
            y_val_batch.to(device)
            y_hat = model(x_val_batch)
            val_loss = loss_fn(y_hat, x_val_batch)

print(f"Model state: {model.state_dict()}")
print(f"Loss is {loss}, Val_Loss is {val_loss}")

Model state: OrderedDict([('a', tensor([2.6968])), ('b', tensor([6.0031])), ('c', tensor([-0.3820]))])
Loss is 0.4589165449142456, Val_Loss is 36.56239318847656


## Для c это относительно точное значение, чтобы получить такую же синусоиду